# Theano  1

Interest in neural networks, and in particular those with architechures that support deep learning, has been surging in recent years.  

In this notebook we will be revisiting the problem of digit classification on the MNIST data.  In doing so, we will introduce a new Python library, Theano, for working with neural networks.  Theano is a popular choice for neural networks as the same code can be run on either CPUs or GPUs.  GPUs greatly speed up the training and prediction, and is readily available. Amazon even offers GPU machines on EC2.  

In part 1, we'll introduce Theano, and refresh ourselves on the MNIST dataset.  In part 2, we'll create a multi-layer neural network with a simple architechure, and train it using backpropagation.  Part 3 will introduce the convolutional architechure, which can be said to be doing 'deep learning' (also called feature learning or representation learning).

#### Part 1: Basics

Lets start to look at Theano.  If later you'd like to go deeper into Theano, you may want to read this paper: http://www.iro.umontreal.ca/~lisa/pointeurs/theano_scipy2010.pdf

Install Theano if you haven't already.  Then let's load it, and set it to work with a CPU.  For reference, here is the Theano documentation: http://www.deeplearning.net/software/theano/library/

In [11]:
%matplotlib inline

import numpy as np
from sklearn.datasets import fetch_mldata
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
import time

import theano 
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
print theano.config.device # We're using CPUs (for now)
print theano.config.floatX # Should be 64 bit for CPUs

np.random.seed(0)

from IPython.display import display, clear_output 

cpu
float64


In [2]:
## basics: theano multiplication

a = T.scalar()
b = T.scalar()

y = a * b

mult = theano.function(inputs = [a, b], outputs = y)

print mult(1, 2)
print mult(4, 5)

2.0
20.0


In [3]:
# more arbitrary

# symbolic variables
a = T.scalar()
b = T.scalar()
c = T.scalar()
d = T.scalar()

# manipulation of these symbolic guys
z = a * b + c / d

# function actually does the work
complexGuy = theano.function(inputs= [a, b, c, d], outputs = z)

# here it is
print complexGuy(3, 3, 1, 5)

9.2


In [4]:
# shared variables: both symbolic and not...
# the update rules; these will be critical for gradient descent updates

w = theano.shared(np.asarray(0., dtype=theano.config.floatX))

xx = T.scalar()

update = [[w, w + xx]]

addition = theano.function(inputs=[xx], outputs=xx, updates=update)

# here's how you actually get it out
print w.get_value()

0.0


In [5]:
# try running this cell many times; observe what update does

addition(1)
print w.get_value()

1.0


In [8]:
# again, try running this many times

addition(4)
print w.get_value()
addition(3)
addition(2)
print w.get_value()

14.0
19.0


In [15]:
# here's an implementation of linear regression

target_beta_1 = 4.
learning_rate = 0.01
noise = 0.2

trainX = np.linspace(-1, 1, 50)
trainY = target_beta_1 * trainX + np.random.randn(*trainX.shape) * noise

X = T.scalar()
Y = T.scalar()

def model(X, w):
    return X * w

w = theano.shared(np.asarray(0., dtype=theano.config.floatX))
yhat = model(X, w)

cost = T.mean(T.sqr((yhat - Y))) 
gradient = T.grad(cost=cost, wrt=w)

updates = [[w, w - gradient * learning_rate]]

train = theano.function(inputs=[X, Y], outputs=cost, updates=updates, allow_input_downcast=True)

for i in range(50):
    for xx, yy in zip(trainX, trainY):
        cc = train(xx, yy)
    clear_output(wait=True)
    print "Round ", i, ":", w.get_value(), cc
        
print "final value: ", w.get_value()

Round  49 : 4.03169743945 0.0578000838739
final value:  4.03169743945


Can we add an intercept to the model above?

In [16]:
## IMPLEMENT: try adding an intercept to the above model

## YOUR CODE HERE

Now for MNIST data...

In [17]:
# Repeating steps from Project 1 to prepare mnist dataset. 
mnist = fetch_mldata('MNIST original', data_home='~/datasets/mnist')
X, Y = mnist.data, mnist.target
X = X / 255.0
shuffle = np.random.permutation(np.arange(X.shape[0]))
X, Y = X[shuffle], Y[shuffle]
numExamples = 2000
test_data, test_labels = X[70000-numExamples:], Y[70000-numExamples:]
train_data, train_labels = X[:numExamples], Y[:numExamples]
numFeatures = train_data[1].size
numTrainExamples = train_data.shape[0]
numTestExamples = test_data.shape[0]
print 'Features = %d' %(numFeatures)
print 'Train set = %d' %(numTrainExamples)
print 'Test set = %d' %(numTestExamples)

Features = 784
Train set = 2000
Test set = 2000


Looking ahead to working with neural networks, let's prepare one additional variation of the label data.  Let's make these labels, rather than each being an integer value from 0-9, be a set of 10 binary values, one for each class.  This is sometimes called a 1-of-n encoding, and it makes working with Neural Networks easier, as there will be one output node for each class.

In [18]:
def binarizeY(data):
    binarized_data = np.zeros((data.size,10))
    for j in range(0,data.size):
        feature = data[j:j+1]
        i = feature.astype(np.int64) 
        binarized_data[j,i]=1
    return binarized_data

train_labels_b = binarizeY(train_labels)
test_labels_b = binarizeY(test_labels)
numClasses = train_labels_b[1].size

print 'Classes = %d' %(numClasses)

print '\n', train_labels_b[:10, :], '\n'
print train_labels[:10], '\n'

Classes = 10

[[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]] 

[ 2.  7.  2.  7.  9.  7.  3.  9.  1.  9.] 



Lets start with a KNN model to establish a baseline accuracy.

Exercise: You've seen a number of different classification algorithms (e.g. naive bayes, decision trees, random forests, logistic regression) at this point.  How does KNN scalability and accuracy with respect to the size of the training dataset compare to those other algorithms?  

In [19]:
neighbors = 1

knn = KNeighborsClassifier(neighbors)
# we'll be waiting quite a while if we use 60K examples, so let's cut it down.  
# You may want to run the full 60K on your own later to see what the accuracy is.
mini_train_data, mini_train_labels = X[:numExamples], Y[:numExamples] 
start_time = time.time()
knn.fit(mini_train_data, mini_train_labels)

print 'Train time = %.2f' %(time.time() - start_time)

start_time = time.time()
accuracy = knn.score(test_data, test_labels)

print 'Accuracy = %.4f' %(accuracy)
print 'Prediction time = %.2f' %(time.time() - start_time)

Train time = 0.10
Accuracy = 0.9095
Prediction time = 6.75


Alright, now that we have a simple baseline, let's start working in Theano.  Before we jump to multi-layer neural networks though, let's train a logistic regression model to make certain we're using Theano correctly. 

Recall from Josh's regression lecture the four key components: (1) parameters, (2) model, (3) cost function, and (4) objective. 

In [20]:
## (1) Parameters 
# Initialize the weights to small, but non-zero, values.
w = theano.shared(np.asarray((np.random.randn(*(numFeatures, numClasses))*.01)))

print (numFeatures, numClasses)

(784, 10)


Two notes relevant at this point:

First, logistic regression can be thought of as a neural network with no hidden layers. The output values are just the dot product of the inputs and the edge weights.

Second, we have 10 classes. We can either train separate one vs all classifiers using sigmoid activation, which would be a hassle, or we can use the softmax activation, which is essentially a multi-class version of sigmoid. We'll use Theano's built-in implementation of softmax.

$$x\in \mathbb{R}^n$$

$$\sigma(x)_j = \frac{e^{x_j}}{\sum_{i=0}^n e^{x_i}}$$

In [21]:
## (2) Model
# Theano objects accessed with standard Python variables
X = T.matrix()
Y = T.matrix()

def model(X, w):
    return T.nnet.softmax(T.dot(X, w))

y_hat = model(X, w)

We'll use cross-entropy as a cost function.  Cross entropy only considers the error between the true class and the prediction, and not the errors for the false classes.  This tends to cause the network to converge faster.  We'll use Theano's built-in cross entropy function.

In [22]:
## (3) Cost function
#cost = T.mean(T.sqr(y_hat - Y))
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))

The objective is minimize the cost, and to do that we'll use batch gradient descent.

Exercise: What are the differences between batch, stochastic, and mini-batch gradient descent?  What are the implications of each for working on large datasets?

We'll use Theano's built-in gradient function.  Exercise: Do you recall from Josh's lecture what the gradient is for beta in logistic regression?

In [23]:
## (4) Objective (and solver)

alpha = 0.01

gradient = T.grad(cost=cost, wrt=w) 
update = [[w, w - gradient * alpha]]

train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True) 
# computes cost, then runs update

y_pred = T.argmax(y_hat, axis=1) # select largest probability as prediction
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

def gradientDescent(epochs):
    trainTime = 0.0
    predictTime = 0.0
    for i in range(epochs):
        start_time = time.time()
        
        cc = train(train_data[0:len(train_data)], train_labels_b[0:len(train_data)])
        trainTime =  trainTime + (time.time() - start_time)
        
        clear_output(wait=True)
        print '%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data)))
    print 'train time = %.2f' %(trainTime)

gradientDescent(500)

start_time = time.time()
predict(test_data)   
print 'predict time = %.2f' %(time.time() - start_time)

500) accuracy = 0.8290
train time = 5.23
predict time = 0.00


Exercise:  What do you expect to happen if we convert batch gradient descent to stochastic gradient descent?  Why?

Let's try it...

In [24]:
## (1) Parameters
w = theano.shared(np.asarray((np.random.randn(*(numFeatures, numClasses))*.01)))

## (2) Model
X = T.matrix()
Y = T.matrix()
def model(X, w):
    return T.nnet.softmax(T.dot(X, w))
y_hat = model(X, w)

## (3) Cost
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))

## (4) Objective
alpha = 0.01
gradient = T.grad(cost=cost, wrt=w)
update = [[w, w - gradient * alpha]] 
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True) 
y_pred = T.argmax(y_hat, axis=1) 
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

miniBatchSize = 1
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):       
        for start, end in zip(range(0, len(train_data), miniBatchSize), range(miniBatchSize, len(train_data), miniBatchSize)):
            cc = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        clear_output(wait=True)
        print '%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data)))  
        
    print 'train time = %.2f' %(trainTime)
    
gradientDescentStochastic(50)

start_time = time.time()
predict(test_data)   
print 'predict time = %.2f' %(time.time() - start_time)

50) accuracy = 0.8735
train time = 255.08
predict time = 0.00


Exercise: What do you expect to happen if you switch the batch size to be greater than 1 (i.e. mini-batch)?  Why?

Try it for a few values...

In [19]:
## (1) Parameters
w = theano.shared(np.asarray((np.random.randn(*(numFeatures, numClasses))*.01)))

## (2) Model
X = T.matrix()
Y = T.matrix()
def model(X, w):
    return T.nnet.softmax(T.dot(X, w))
y_hat = model(X, w)

## (3) Cost
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))

## (4) Objective
alpha = 0.01
gradient = T.grad(cost=cost, wrt=w)
update = [[w, w - gradient * alpha]] 
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True) 
y_pred = T.argmax(y_hat, axis=1) 
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

miniBatchSize = 10 
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):       
        for start, end in zip(range(0, len(train_data), miniBatchSize), range(miniBatchSize, len(train_data), miniBatchSize)):
            cc = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        clear_output(wait=True)
        print '%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data))) 
        
    print 'train time = %.2f' %(trainTime)
    
gradientDescentStochastic(50)

start_time = time.time()
predict(test_data)   
print 'predict time = %.2f' %(time.time() - start_time)

50) accuracy = 0.8825
train time = 38.94
predict time = 0.01
